In [334]:
from pyscopus import Scopus
import pandas as pd
import requests
import xmltodict, json


from lxml import etree
import requests
import xmltodict, json

In [335]:
print("Ingrese el dato a buscar:")
busqueda = input()

Ingrese el dato a buscar:
medicine


In [336]:
#Consumo API de SCOPUS

In [337]:
key = '0387c7142db3254ed53c09c131ecf8c0'
scopus = Scopus(key)
search_df = scopus.search("ALL("+busqueda+")", count=100)

In [338]:
#Consumo API de DOAJ

In [339]:
url = 'https://www.doaj.org/api/v2/search/articles/all('+busqueda+')?pageSize=100'
headers = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
r = requests.get(url, headers=headers)

In [340]:
#Consumo API de propiedad Inletectual

In [341]:
url_uspto = 'https://developer.uspto.gov/ibd-api/v1/application/publications?searchText='+busqueda+'&start=0&rows=100'
headers_uspto = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
r2_uspto = requests.get(url_uspto, headers=headers_uspto) 

In [342]:
#Consumo API deProQuest

In [343]:
access_token = '8461684a-5a79-407c-9321-93d8d719ddeb'
headers = {
    'Content-type': 'text/xml',
    'Authorization': 'Bearer {}'.format(access_token)
}
data = '<searchRequest><search><query>'+busqueda+'</query><databases><database>medlineprof</database></databases></search><count>100</count></searchRequest>'
resp = requests.post('https://api-dialog.proquest.com/v1/search', headers=headers, data=data)
o = xmltodict.parse(resp.text)
conta = json.dumps(o)
d = json.loads(conta)


In [344]:
#Declaracion 
listalinks = list()
listatitulos = list()
listabstract = list()
listautor= list()
listakeywords = list()
listafuente = list()


In [345]:
#Consumo api  de propiedad intelectual sobre titulo,url,abstract,nombre(uspto)
content_uspto = r2_uspto.json()
json_uspto = content_uspto['results']

for i in range(len(json_uspto)):
    try:

        listakeywords.append(0)
        listafuente.append("uspto")
        
        nivel1=json_uspto[i]['inventionTitle']
        listatitulos.append(nivel1)
        
        nivel2=json_uspto[i]['filelocationURI']
        listalinks.append(nivel2)
        
        nivel3=json_uspto[i]['abstractText'][0]
        listabstract.append(nivel3)
        
        nivel4=json_uspto[i]['assigneeEntityName']
        listautor.append(nivel4)
    except:
        print(" ")


In [346]:
#Consumo api  de doaj sobre titulo,url,abstract,author(uspto),keywords
content = r.json()
#Ingreso al contenido
json = content['results']

for i in range(len(json)):
    try:

        nivel4=json[i]['bibjson']['keywords']
        listakeywords.append(nivel4)
        nivel1=json[i]['bibjson']['link'][0]['url']
        listalinks.append(nivel1)
        nivel2=json[i]['bibjson']['title']
        listatitulos.append(nivel2)
        nivel3=json[i]['bibjson']['abstract']
        listabstract.append(nivel3)
        nivel4=json[i]['bibjson']['author']
        listautor.append(nivel4)
        listafuente.append("doaj")
        
    except:
        print("")

In [347]:
#Consumo api  de SCOPUS sobre titulo,url,abstract,nombre(uspto)
for i in range(len(search_df['scopus_id'])):
    try:
        pub_abs = scopus.retrieve_abstract(search_df['scopus_id'][i])
        listabstract.append(pub_abs['abstract'])
        listalinks.append("https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp="+search_df['scopus_id'][i]+"&origin=inward")
        listatitulos.append(search_df['title'][i])
        listautor.append(search_df['authors'][i])
        listakeywords.append("0")
        listafuente.append("scopus")
    except:
        print(" ")




In [348]:
#Consumo api  de ProQuest sobre titulo,url,abstract,nombre(uspto)
json = d['searchResponse']['result']['doc']

for i in range(len(json)):

        #TITULO
        nivel1=json[i]['field'][1]['value']
        listatitulos.append(nivel1)
        nivel2=json[i]['field'][3]['value']
        response2 = requests.get(nivel2, headers=headers)
        o1 = xmltodict.parse(response2.text)
        
        #LINK
        listalinks.append(o1['Documents']['Literature']['DocInfo']['URL'])
        
        #Fuente
        listafuente.append('ProQuest')
        
        lista
        #Authores
        try:
            
            contributor=o1['Documents']['Literature']['Contributors']['Contributor']
            aux=list()
            for i in range(len(contributor)):
                #print(contributor[i])
                if(contributor[i]['@ContribRole']=='Author'):
                    aux.append(contributor[i]['NormalizedName']+" "+contributor[i]['LastName'])
            listautor.append(aux)
        except: 
             listautor.append("NULL")

        #Keywords
        keywords=o1['Documents']['Literature']['Subjects']['HeadingTerms']['HeadingTerm']
        aux=list()
        for i in range(len(keywords)):
            nivel9=keywords[i]['Heading']
            aux.append(nivel9['#text'])
        listakeywords.append(aux)
        
        #ABSTRACT
        try:
            nivel5= o1['Documents']['Literature']['Abstract']
            listabstract.append(nivel5.get('#text'))
        except: 
            listabstract.append("NULL")

In [349]:
diccionario ={'link':listalinks ,'title':listatitulos,'abstract':listabstract,'authors':listautor,"keywords":listakeywords,'fuente':listafuente}

In [350]:
dataset=pd.DataFrame(diccionario)
print(dataset)

                                                  link  \
0    https://developer-hub.s3.amazonaws.com/publica...   
1    https://developer-hub.s3.amazonaws.com/publica...   
2    https://developer-hub.s3.amazonaws.com/publica...   
3    https://developer-hub.s3.amazonaws.com/publica...   
4    https://developer-hub.s3.amazonaws.com/publica...   
..                                                 ...   
358  http://dialog.proquest.com/professional/docvie...   
359  http://dialog.proquest.com/professional/docvie...   
360  http://dialog.proquest.com/professional/docvie...   
361  http://dialog.proquest.com/professional/docvie...   
362  http://dialog.proquest.com/professional/docvie...   

                                                 title  \
0    ULTRA-SMALL ICE, USES THEREOF AND APPARATUS FO...   
1                    Apparatus for Producing Flake Ice   
2       Device for Controlling Leaks From an Endoscope   
3    DELIVERY DEVICE FOR DISPENSING PHARMACEUTICAL ...   
4       LIQUI

In [351]:
dataset.to_csv('resultado.csv')